# GOAL: 다른 테이블 항생제 가져오기 INTIME<= 항생제 <=OUTTIME

In [1]:
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
#필요한 패키지들 import하기
import pandas as pd
# import modin.pandas as pd
import numpy as np
from tqdm import tqdm
import os
import re
import gzip
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
icu_path = '/data2/je_hong/kaist/kaist/physionet.org/files/mimiciv/2.0/icu/'

In [3]:
new_path = '/data2/je_hong/kaist/kaist/physionet.org/files/mimiciv/2.0/mimic_rev/'

In [4]:
hos_path = '/data2/je_hong/kaist/kaist/physionet.org/files/mimiciv/2.0/hosp/'

In [5]:
import pickle
with open(new_path+'new_dict/adm_stay.pkl','rb') as f:
    dict_ = pickle.load(f)



In [6]:
prescriptions_ab = pd.read_csv(new_path+'new_csv/antibio_nonenc.csv')

In [7]:
prescriptions_ab.head()

,subject_id,hadm_id,stay_id,icu_intime,icu_outtime,drug,ab_starttime,ab_stoptime
0,12466550,23998182,30000153,2174-09-29 12:09:00,2174-10-01 03:26:10,penicillins,2174-10-04 10:00:00,2174-10-04 19:00:00
1,18421337,22413411,30000484,2136-01-14 17:23:32,2136-01-17 04:53:08,fluoroquinolones,2136-01-14 21:00:00,2136-01-15 16:00:00
2,12207593,22795209,30000646,2194-04-29 01:39:22,2194-05-03 18:23:48,glycopeptides,2194-04-29 07:00:00,2194-04-30 06:00:00
3,12980335,23552849,30001148,2156-08-30 11:10:59,2156-08-31 14:25:34,glycopeptides,2156-08-30 20:00:00,2156-08-31 15:00:00
4,17371178,24502166,30001396,2147-10-18 12:48:00,2147-10-19 16:49:23,glycopeptides,2147-10-20 07:00:00,2147-10-21 20:00:00


In [8]:
HADMS = list(dict_.keys())

In [9]:
stay_ids = []
for i in dict_.keys():
    stay_ids.append(dict_[i][0]['STAYID'])

In [10]:
HADMS

[24903681,
 24510466,
 27262979,
 25559044,
 25034757,
 29622279,
 21889037,
 26214416,
 26869778,
 27000850,
 20709396,
 23724055,
 27131937,
 24903714,
 24248353,
 27918373,
 28311590,
 24903719,
 21258095,
 20185129,
 22544425,
 22413355,
 20185136,
 27787312,
 21258097,
 26738740,
 28049461,
 25690167,
 22675517,
 21102653,
 26083391,
 26083390,
 25428033,
 25296963,
 26476613,
 23724101,
 23855176,
 28966984,
 24510536,
 24248394,
 29098061,
 27525200,
 20840530,
 22282324,
 26476632,
 27525208,
 29360220,
 25559133,
 20054108,
 27263071,
 24248416,
 24117340,
 24510562,
 20185187,
 26607715,
 27263076,
 22413411,
 29229158,
 21364840,
 26476649,
 24379499,
 27918444,
 22020207,
 23593072,
 24903793,
 28967026,
 25952371,
 23462007,
 23855224,
 28573817,
 29884542,
 27787392,
 22020231,
 28442760,
 24379528,
 20185229,
 29622413,
 24641678,
 20971664,
 26345618,
 23593110,
 24772758,
 21758109,
 24772768,
 22806688,
 28311712,
 25952422,
 23462055,
 20578475,
 22413484,
 23855276,

In [11]:
prescriptions_ab.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'icu_intime', 'icu_outtime', 'drug',
       'ab_starttime', 'ab_stoptime'],
      dtype='object')

In [12]:
ab_time=prescriptions_ab[['hadm_id','drug','ab_starttime','ab_stoptime']]

In [13]:
# ab_time['hadm_id'] = ab_time['hadm_id'].apply(lambda x:np.nan if x in HADMS else x)

In [14]:
from datetime import datetime

NOT_CONVERTED = 'NOT_CONVERTED'


def str2datetime(s):
    def _convert(_s, _dformat):
        try:
            converted_dt = datetime.strptime(_s, _dformat)
        except Exception:
            return NOT_CONVERTED

        return converted_dt

    if isinstance(s, datetime):
        return s

    dformats = [
        '%Y-%m-%d %p %I:%M:%S',
        '%Y-%m-%d %H:%M:%S.%f',
        '%Y-%m-%d %H:%M:%S',
        '%Y-%m-%d %H:%M',
        '%y-%m-%d %H:%M',
        '%Y-%m-%d',
        '%Y%m%d',
        '%Y-%m-%d +%H:%M',
        '%Y%m%d %H%M',
        '%Y%m%d%H%M%S',
        ]

    s = s.split('.')[0]
    for dformat in dformats:
        dt = _convert(s, dformat)
        if dt != NOT_CONVERTED:
            return dt

    return NOT_CONVERTED

In [15]:
drop_ab_time = ab_time.dropna(subset=['hadm_id'])

In [16]:
drop_ab_time

,hadm_id,drug,ab_starttime,ab_stoptime
0,23998182,penicillins,2174-10-04 10:00:00,2174-10-04 19:00:00
1,22413411,fluoroquinolones,2136-01-14 21:00:00,2136-01-15 16:00:00
2,22795209,glycopeptides,2194-04-29 07:00:00,2194-04-30 06:00:00
3,23552849,glycopeptides,2156-08-30 20:00:00,2156-08-31 15:00:00
4,24502166,glycopeptides,2147-10-20 07:00:00,2147-10-21 20:00:00
...,...,...,...,...
48189,22341980,penicillins,2158-03-05 11:00:00,2158-03-06 11:00:00
48190,20178444,macrolides,2111-08-19 14:00:00,2111-08-19 23:00:00
48191,27161460,cephalosporins,2158-05-24 21:00:00,2158-05-25 03:00:00
48192,26910235,cephalosporins,2186-07-17 14:00:00,2186-07-18 18:00:00


In [17]:
## data type 변경
drop_ab_time['ab_starttime'] = pd.to_datetime(drop_ab_time['ab_starttime'])
drop_ab_time['ab_stoptime'] = pd.to_datetime(drop_ab_time['ab_stoptime'])
# drop_chartevents['outtime'] = pd.to_datetime(drop_chartevents['outtime'])

In [18]:
drop_ab_time.columns

Index(['hadm_id', 'drug', 'ab_starttime', 'ab_stoptime'], dtype='object')

In [19]:
for k in dict_.keys():
    
    intime = (dict_[k][0]['INTIME'])
    outtime = (dict_[k][0]['OUTTIME'])
    temp_df = drop_ab_time[drop_ab_time['hadm_id'] == k]
    outtime = (dict_[k][0]['OUTTIME'])
    

    temp_df = temp_df.sort_values(by = ['ab_starttime'], ascending=True)
    starttime = temp_df['ab_starttime']
    stoptime = temp_df['ab_stoptime']
    drug = temp_df['drug'].values
    try:
        print(drug[0])
    
        if str2datetime(str(intime)) <= str2datetime(str(starttime)) <= str2datetime(str(outtime)):
            if str2datetime(str(intime)) <= str2datetime(str(stoptime)) <= str2datetime(str(outtime)):
                dict_[k][0]['DRUG'] = drug
    except:
        pass

            
#     print(temp_df['starttime'])
#     starttime = temp_df['starttime'].values

macrolides
penicillins
glycopeptides
nitroimidazole 
fluoroquinolones
macrolides
cephalosporins
cephalosporins
glycopeptides
glycopeptides
fluoroquinolones
sulfonamides(diaminopyrimidine)
cephalosporins
cephalosporins
cephalosporins
fluoroquinolones
cephalosporins
glycopeptides
glycopeptides
cephalosporins
penicillins
cephalosporins
cephalosporins
sulfonamides(diaminopyrimidine)
glycopeptides
nitroimidazole 
penicillins
cephalosporins
fluoroquinolones
penicillins
glycopeptides
cephalosporins
glycopeptides
macrolides
glycopeptides
penicillins
cephalosporins
penicillins
fluoroquinolones
cephalosporins
fluoroquinolones
cephalosporins
glycopeptides
nitroimidazole 
penicillins
cephalosporins
macrolides
penicillins
cephalosporins
macrolides
glycopeptides
penicillins
nitroimidazole 
cephalosporins
macrolides
cephalosporins
cephalosporins
cephalosporins
cephalosporins
cephalosporins
glycopeptides
cephalosporins
glycopeptides
cephalosporins
nitroimidazole 
penicillins
glycopeptides
macrolides
s

In [21]:
dict_[k][0]

{'INTIME': numpy.datetime64('2175-04-15T11:28:30.000000000'),
 'OUTTIME': numpy.datetime64('2175-04-28T20:05:08.000000000'),
 'IS_DEAD': 0,
 'DOD': numpy.datetime64('NaT'),
 'STAYID': 32035918,
 'DRUG': array(['cephalosporins'], dtype=object)}

In [22]:
import pickle
with open(new_path+'new_dict/adm_stay_feature.pkl','wb') as f:
    pickle.dump(dict_,f)
